<a href="https://colab.research.google.com/github/balajiceg/EE_TF_Vembanad/blob/master/vembanad_EEIA2019_EE_%2B_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This is an Earth Engine <> TensorFlow demonstration notebook.  Specifically, this notebook shows:

1.   Using training/testing data exported from Earth Engine in TFRecord format.
2.   Training and validating a simple model (Keras `Sequential` neural network) in TensorFlow.
3.   Making predictions on image data exported from Earth Engine in TFRecord format.
4.   Importing classified image data to Earth Engine in TFRecord format.

# Install the Earth Engine client library

This only needs to be done once per notebook.

In [0]:
!pip install earthengine-api

# Authentication

To read/write from a Google Cloud Storage bucket to which you have access, it's necessary to authenticate (as yourself).  You'll also need to authenticate as yourself with Earth Engine, so that you'll have access to your scripts, assets, etc.

## Authenticate to Colab and Cloud

Identify yourself to Google Cloud, so you have access to storage and other resources.  When you run the code below, it will display a link in the output to an authentication page in your browser.  Follow the link to a page that will let you grant permission to the Cloud SDK to access your resources.  Copy the code from the permissions page back into this notebook and press return to complete the process.

(You may need to run this again if you get a credentials error later.)

In [0]:
from google.colab import auth

auth.authenticate_user()

## Authenticate to Earth Engine

Authenticate to Earth Engine the same way you did to the Colab notebook.  Specifically, run the code to display a link to a permissions page.  This gives you access to your Earth Engine account.  Copy the code from the Earth Engine permissions page back into the notebook and press return to complete the process.

In [0]:
!earthengine authenticate

# Test the software setup

## Test the Earth Engine installation



In [0]:
# Import the Earth Engine API and initialize it.
import ee
ee.Initialize()

# Test the earthengine command by getting help on upload.
!earthengine upload image -h

## Test the TensorFlow installation

The default public runtime already has the tensorflow libraries we need installed.  Before any operations from the TensorFlow API are used, import TensorFlow and enable eager execution.  This provides an imperative interface that can help with debugging.  See the [TensorFlow eager execution guide](https://www.tensorflow.org/guide/eager) or the [`tf.enable_eager_execution()` docs](https://www.tensorflow.org/api_docs/python/tf/enable_eager_execution) for details. 

In [0]:
import tensorflow as tf

tf.enable_eager_execution()
print(tf.__version__)

# Data preparation and pre-processing

Read data from the TFRecord file into a `tf.data.Dataset`.  Pre-process the dataset to get it into a suitable format for input to the model.

## Read into a `tf.data.Dataset`

Here we are going to read a file in Cloud Storage into a `tf.data.Dataset`.  ([these TensorFlow docs](https://www.tensorflow.org/guide/premade_estimators#create_input_functions) explain more about reading data into a `Dataset`).  Check that you can read examples from the file.  The purpose here is to ensure that we can read from the file without an error.  The actual content is not necessarily human readable.



In [0]:
#link gdrive to import the tfrecord
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [0]:
# Create a dataset from the TFRecord file in Cloud Storage.
from pprint import pprint
trainDataset = tf.data.TFRecordDataset("/gdrive/My Drive/vembanad_samples_1218/vembanad_samples_1218.gz", compression_type='GZIP')
# Print the first record to check.
pprint(iter(trainDataset).next())

## Define the structure of your data

For parsing the exported TFRecord files, `featuresDict` is a mapping between feature names (recall that `featureNames` contains the band and label names) and `float32` [`tf.io.FixedLenFeature`](https://www.tensorflow.org/api_docs/python/tf/io/FixedLenFeature) objects.  This mapping is necessary for telling TensorFlow how to read data in a TFRecord file into tensors.  Specifically, all numeric data exported from Earth Engine is exported as `float32`. 

(Note: *features* in the TensorFlow context (i.e. [`feature.proto`](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/core/example/feature.proto)) are not to be confused with Earth Engine features (i.e. [`ee.Feature`](https://developers.google.com/earth-engine/api_docs#eefeature)), where the former is a protocol message type for serialized data input to the model and the latter is a geometry-based geographic data structure.)  

In [0]:
# List of fixed-length features, all of which are float32.
# featureNames=[ "to_settle",  "b2",  "ndvi_2018",  "b3",  "b4",  "b5",  "b6",  "ndvi_2012",  "b1"];
featureNames=[ "to_settle",  "b2",  "b3",  "b4",  "b6","b5",  "b1"];
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in featureNames
]

# Dictionary with names as keys, features as values.
featuresDict = dict(zip(featureNames, columns))

pprint(featuresDict)

## Parse the dataset

Now we need to make a parsing function for the data in the TFRecord files.  The data comes in flattened 2d arrays per record and we want to use the first part of the array for input to the model and the last element of the array as the class label.  The parsing function reads data from a serialized `Example` proto (i.e. [`example.proto`](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/core/example/example.proto)) into a dictionary in which the keys are the feature names and the values are the tensors storing the value of the features for that example.  ([Learn more about parsing `Example` protocol buffer messages](https://www.tensorflow.org/programmers_guide/datasets#parsing_tfexample_protocol_buffer_messages)).  The parsing function uses the `featuresDict` to read the serialized example, then returns a tuple of the predictors and the label, cast to an `int32`.  The check at the end is to print a single record from the parsed TFRecord file.

In [0]:
# Define the parsing function.
label = 'to_settle'
def parse_tfrecord(example_proto):
  parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
  labels = parsed_features.pop(label)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsedDataset = trainDataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first thing to check.
pprint(iter(parsedDataset).next())

Note that each record of the parsed dataset contains a tuple.  The first element of the tuple is a dictionary with bands for keys and the numeric value of the bands for values.  The second element of the tuple is a class label.

# Model setup

The basic workflow for classification in TensorFlow is:

1.  Create the model.
2.  Train the model (i.e. `fit()`).
3.  Use the trained model for inference (i.e. `predict()`).  

Here we'll create a `Sequential` neural network model using Keras.  This simple model is inpired by examples in:

* [The TensorFlow Get Started tutorial](https://www.tensorflow.org/tutorials/)
* [The TensorFlow Keras guide](https://www.tensorflow.org/guide/keras#build_a_simple_model)
* [The Keras `Sequential` model examples](https://keras.io/getting-started/sequential-model-guide/#multilayer-perceptron-mlp-for-multi-class-softmax-classification)

Note that the model demonstrated here is purely for demonstration purposes and hasn't gone through any performance tuning.

## Create the Keras model

Before we create the model, there's still a wee bit of pre-processing to get the data into the right input shape and a format that can be used with cross-entropy loss.  Specifically, Keras expects a list of inputs and a one-hot vector for the class.(See [the Keras loss function docs](https://keras.io/losses/), [the TensorFlow categorical identity docs](https://www.tensorflow.org/guide/feature_columns#categorical_identity_column) and [the `tf.one_hot` docs](https://www.tensorflow.org/api_docs/python/tf/one_hot) for details).  

Here we will use a simple neural network model with a 64 node hidden layer, a dropout layer and an output layer.  Once the dataset has been prepared, define the model, compile it, fit it to the training data.  See [the Keras `Sequential` model guide](https://keras.io/getting-started/sequential-model-guide/) for more details.


In [0]:
from tensorflow import keras

# How many classes there are in the model.
nClasses = 2
# Add NDVI.
inputDataset = parsedDataset
# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def toTuple(dict, label):
  return tf.transpose(dict.values()), tf.one_hot(indices=label-1, depth=nClasses)

# Repeat the input dataset as many times as necessary in batches of 10.
inputDatasetTuple = inputDataset.map(toTuple)

#split into train test vaidation
DATASET_SIZE = [i for i,_ in enumerate(inputDatasetTuple)][-1] + 1

inputDatasetTuple=inputDatasetTuple.shuffle(DATASET_SIZE+1)
train_size = int(0.6 * DATASET_SIZE)
val_size = int(0.20 * DATASET_SIZE)
test_size = int(0.20 * DATASET_SIZE)

trainDataset = inputDatasetTuple.take(train_size).repeat().batch(10)
testDataset = inputDatasetTuple.skip(train_size)
valDataset = testDataset.skip(val_size).repeat().batch(10)
testDataset = testDataset.batch(1)


# pprint(iter(inputDatasetTuple).next())

 tf.keras.layers.Dense(24,kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
  tf.keras.layers.ReLU(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  
  tf.keras.layers.Dense(8,kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
  tf.keras.layers.ReLU(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),

In [0]:
learning_rate=0.001
steps_per_epoch=15

# Define the layers in the model.
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64,kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
  tf.keras.layers.ReLU(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  
#   tf.keras.layers.Dense(32,kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
#   tf.keras.layers.ReLU(),
#   tf.keras.layers.BatchNormalization(),
#   tf.keras.layers.Dropout(0.2),
    
#   tf.keras.layers.Dense(8,kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
#   tf.keras.layers.ReLU(),
#   tf.keras.layers.BatchNormalization(),
#   tf.keras.layers.Dropout(0.2),
    
  tf.keras.layers.Dense(nClasses, activation=tf.nn.softmax)  
])

# Compile the model with the specified loss function.
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model to the training data.
# Don't forget to specify `steps_per_epoch`  when calling `fit` on a dataset.
history=model.fit(x=trainDataset, epochs=100, steps_per_epoch=steps_per_epoch,validation_data=valDataset,validation_steps=10)


Plotting the training and validation curves

In [0]:
import matplotlib.pyplot as plt
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Check model accuracy on the test set

Now that we have a trained model, we can evaluate it using the test dataset.  To do that, read and prepare the test dataset in the same way as the training dataset.  Here we specify a batch sie of 1 so that each example in the test set is used exactly once to compute model accuracy.  For model steps, just specify a number larger than the test dataset size (ignore the warning).

In [0]:
model.evaluate(testDataset, steps=100)

# Use the trained model to classify an image from Earth Engine

Now it's time to classify the image that was exported from Earth Engine.  If the exported image is large, it will be split into multiple TFRecord files in its destination folder.  There will also be a JSON sidecar file called "the mixer" that describes the format and georeferencing of the image.  Here we will find the image files and the mixer file, getting some info out of the mixer that will be useful during model inference.

## Find the image files and JSON mixer file in Cloud Storage

Use `gsutil` to locate the files of interest in the output Cloud Storage bucket.  Check to make sure your image export task finished before running the following.

In [0]:
# Get a list of all the files in the output bucket.
imageFilePrefix='vembanad_image_stn_1218'
import glob
filesList=glob.glob("/gdrive/My Drive/vembanad_samples_1218/*")

# Get only the files generated by the image export.
exportFilesList = [s for s in filesList if imageFilePrefix in s]

# Get the list of image files and the JSON mixer file.
imageFilesList = []
jsonFile = None
for f in exportFilesList:
  if f.endswith('.gz'):
    imageFilesList.append(f)
  elif f.endswith('.json'):
    jsonFile = f

# Make sure the files are in the right order.
imageFilesList.sort()

pprint(imageFilesList)
print(jsonFile)

## Read the JSON mixer file

The mixer contains metadata and georeferencing information for the exported patches, each of which is in a different file.  Read the mixer to get some information needed for prediction.

In [0]:
import json

# Load the contents of the mixer file to a JSON object.
jsonText = !cat '{jsonFile}'
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(jsonText.nlstr)
pprint(mixer)

## Read the image files into a dataset

You can feed the list of files (`imageFilesList`) directly to the `TFRecordDataset` constructor to make a combined dataset on which to perform inference.  The input needs to be preprocessed differently than the training and testing.  Mainly, this is because the pixels are written into records as patches, we need to read the patches in as one big tensor (one patch for each band), then flatten them into lots of little tensors.  We also need to add dummy labels.

In [0]:
bands=["b1","b2","b3","b4","b5","b6"]
# Get relevant info from the JSON mixer file.
PATCH_WIDTH = mixer['patchDimensions'][0]
PATCH_HEIGHT = mixer['patchDimensions'][1]
PATCHES = mixer['totalPatches']
PATCH_DIMENSIONS_FLAT = [PATCH_WIDTH * PATCH_HEIGHT, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
imageColumns = [
  tf.FixedLenFeature(shape=PATCH_DIMENSIONS_FLAT, dtype=tf.float32) 
    for k in bands
]

# Parsing dictionary.
imageFeaturesDict = dict(zip(bands, imageColumns))

# Note that you can make one dataset from many files by specifying a list.
imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.parse_single_example(example_proto, imageFeaturesDict)

# Parse the data into tensors, one long tensor per patch.
imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many littler ones.
imageDataset = imageDataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Add additional features (NDVI).
# imageDataset = imageDataset.map(
#   # Add NDVI to a feature that doesn't have a label.
#   lambda features: addFeatures(features, None)[0]
# )

# Turn the dictionary in each record into a tuple with a dummy label.
imageDataset = imageDataset.map(
  # Add a dummy target (-1), with a value that is obviously ridiculous.
  # This is because the model expects a tuple of (inputs, label).
  lambda dataDict: (tf.transpose(dataDict.values()), tf.constant(-1))
)

# Turn each patch into a batch.
imageDataset = imageDataset.batch(PATCH_WIDTH * PATCH_HEIGHT)

## Generate predictions for the image pixels

To get predictions in each pixel, run the image dataset through the trained model using `model.predict()`.  Print the first prediction to see that the output is a list of the three class probabilities for each pixel.  Running all predictions might take a while.

In [0]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(imageDataset, steps=PATCHES, verbose=1)

print(iter(predictions).next())

## Write the predictions to a TFRecord file

Now that there's a list of class probabilities in `predictions`, it's time to write them back into a file, optionally including a class label which is simply the index of the maximum probability.  We'll write directly from TensorFlow to a file in the output Cloud Storage bucket.

Iterate over the list, compute class label and write the class and the probabilities in patches.  Specifically, we need to write the pixels into the file as patches in the same order they came out.  The records are written as serialized `tf.train.Example` protos.  This might take a while.

In [0]:
outputImageFile = '/gdrive/My Drive/vembanad_samples_1218/vembanad_1218_output1'

# Instantiate the writer.
writer = tf.python_io.TFRecordWriter(outputImageFile)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[], [], []]
curPatch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1))
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == PATCH_WIDTH * PATCH_HEIGHT):
    print('Done with patch ' + str(curPatch) + ' of ' + str(PATCHES) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          'nonSetProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'setProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2]))
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[], [], []]
    curPatch += 1

writer.close()

# Upload the classifications to an Earth Engine asset

## Verify the existence of the predictions file

At this stage, there should be a predictions TFRecord file sitting in the output Cloud Storage bucket.  Use the `gsutil` command to verify that the predictions image (and associated mixer JSON) exist and have non-zero size.

In [0]:
# !gsutil ls -l {outputImageFile}
!earthengine upload image -h

## Upload the classified image to Earth Engine

Upload the image to Earth Engine directly from the Cloud Storage bucket with the [`earthengine` command](https://developers.google.com/earth-engine/command_line#upload).  Provide both the image TFRecord file and the JSON file as arguments to `earthengine upload`.

In [0]:
# REPLACE WITH YOUR ASSETS FOLDER!
outputAssetID = 'users/balaji9th/vembanad/new' 
prefix = 'gs://eeia2019/'

# jsonFile = prefix+'Image_pixel_demo_floatmixer.json'
# jsonFile = prefix+'Image_pixel_demo_float00000.tfrecord.gz
# jsonFile = prefix+'Image_pixel_demo_float00001.tfrecord.gz
# jsonFile = prefix+'Image_pixel_demo_float00002.tfrecord.gz


# Start the upload.
# !earthengine upload image --asset_id={outputAssetID} {outputImageFile} {jsonFile}

!earthengine upload image --asset_id={outputAssetID} {outputImageFile}

## Check the status of the asset ingestion

You can also use the Earth Engine API to check the status of your asset upload.  The upload of the image is an asset ingestion task.  

In [0]:
ee.batch.Task.list()

## View the ingested asset



In [0]:
import folium
predictionsImage = ee.Image(outputAssetID)
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

predictionVis = {
  'bands': 'prediction',
  'min': 0,
  'max': 1,
  'palette': ['red', 'green']
}
probabilityVis = {'bands': ['nonSetProb', 'setProb']}

predictionMapid = predictionsImage.getMapId(predictionVis)
probabilityMapid = predictionsImage.getMapId(probabilityVis)

map = folium.Map(location=[38., -122.5])
folium.raster_layers.TileLayer(
  tiles=EE_TILES.format(**predictionMapid),
  attr='Google Earth Engine',
  overlay=True,
  name='prediction',
).add_to(map)
folium.raster_layers.TileLayer(
  tiles=EE_TILES.format(**probabilityMapid),
  attr='Google Earth Engine',
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map